# Compréhension du problème :
On travaille sur un problème de régression, car on essaye de prédire une valeur numérique continue 'SiteEnergyUse(kBtu)', 'TotalGHGEmissions' à partir de caractéristiques de bâtiments.

# Les modèles testés seront les suivants : 

## Linear Regression (Régression linéaire):
La régression linéaire est un modèle de régression simple qui tente de modéliser la relation linéaire entre les caractéristiques (variables prédictives) et la variable cible.
L'objectif est de trouver une ligne droite (ou un hyperplan en plusieurs dimensions) qui minimise l'erreur quadratique moyenne entre les prédictions du modèle et les valeurs réelles de la variable cible.
C'est un modèle simple et facile à interpréter, mais il suppose que la relation entre les caractéristiques et la cible est linéaire, ce qui peut ne pas être vrai dans tous les cas.

## ElasticNet:
ElasticNet est une extension de la régression linéaire qui combine à la fois la régression L1 (Lasso) et la régression L2 (Ridge).
Il est utilisé pour gérer le problème de la multicollinéarité (lorsque les caractéristiques sont fortement corrélées) et pour effectuer la sélection automatique des caractéristiques en réduisant les coefficients de certaines caractéristiques à zéro.
Les paramètres alpha et l1_ratio permettent de contrôler le mélange entre L1 et L2.

## Random Forest Regressor (Régresseur à forêts aléatoires):
Les forêts aléatoires sont des modèles d'ensemble basés sur des arbres de décision.
Dans le cas du régresseur à forêts aléatoires, de nombreux arbres de décision sont construits lors de l'apprentissage et les prédictions finales sont la moyenne des prédictions de chaque arbre.
Ils sont très flexibles et peuvent modéliser des relations complexes entre les caractéristiques et la variable cible. Ils sont également robustes face aux données bruyantes et aux valeurs aberrantes.

## Gradient Boosting Regressor (Régresseur par Gradient Boosting):
Le régresseur par gradient boosting est un autre modèle d'ensemble qui construit un modèle prédictif en ajoutant séquentiellement des arbres de décision faibles.
À chaque étape, le modèle tente de corriger les erreurs du modèle précédent en ajustant les poids des observations mal prédites.
Cela conduit généralement à un modèle très puissant, capable de modéliser des relations complexes. Cependant, il est plus susceptible de sur-ajuster si les hyperparamètres ne sont pas correctement réglés.

## Support Vector Regressor (Régresseur à vecteurs de support):
Le régresseur à vecteurs de support est utilisé pour la régression. Il est basé sur le même principe que la classification SVM, mais il vise à minimiser les erreurs de prédiction pour les données de régression.
Il tente de trouver un hyperplan (ou une surface hyperplane en plusieurs dimensions) qui maximise la marge entre les points de données et la prédiction du modèle.
Il est efficace pour les données de faible dimension, mais peut être moins approprié pour les ensembles de données de grande dimension.

# Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold

# Data

In [2]:
data = pd.read_csv('data_saved.csv')

# Chargement de la liste des targets et features

In [3]:
features = np.load('features.npy')
targets = np.load('targets.npy')

### Création d'une liste de caractéristiques continues et catégorielles pour différencier quelles caractéristiques nécessitent un traitement.

In [5]:
# Initialisation des listes vides pour les caractéristiques continues et catégorielles
features_continuous = []
features_categorical = []

# Boucle à travers les caractéristiques et identification de leur type de données
for feature in features:
    dtype = data[feature].dtype
    if np.issubdtype(dtype, np.number):  # Vérifiez si c'est un type de données numérique
        features_continuous.append(feature)
    else:
        features_categorical.append(feature)

# Affichez les caractéristiques continues et catégorielles
print("Caractéristiques continues :", features_continuous)
print("Caractéristiques catégorielles :", features_categorical)

Caractéristiques continues : ['NumberofBuildings', 'PropertyGFATotal', 'PropertyGFABuilding(s)', 'BuildingAge', 'LargestPropertyUseTypeGFA', 'Electricity(kWh)', 'Electricity(kBtu)']
Caractéristiques catégorielles : ['PrimaryPropertyType']


In [7]:
print(targets)

['SiteEnergyUse(kBtu)' 'TotalGHGEmissions']


In [9]:
print("Caractéristiques continues :", features_continuous)
print("Caractéristiques catégorielles :", features_categorical)

Caractéristiques continues : ['NumberofBuildings', 'PropertyGFATotal', 'PropertyGFABuilding(s)', 'BuildingAge', 'LargestPropertyUseTypeGFA', 'Electricity(kWh)', 'Electricity(kBtu)']
Caractéristiques catégorielles : ['PrimaryPropertyType']


# Pipeline complet avec le préprocesseur et le modèles d'apprentissages supervisés pour target 'SiteEnergyUse(kBtu)'

In [15]:
# Définissez vos caractéristiques X et votre variable cible y
X = data[features]
y = data['SiteEnergyUse(kBtu)']

# Divisez les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Liste des colonnes numériques
numerical_cols = ['NumberofBuildings', 'PropertyGFATotal', 'PropertyGFABuilding(s)', 'BuildingAge', 'LargestPropertyUseTypeGFA', 'Electricity(kWh)', 'Electricity(kBtu)']

# Liste des colonnes catégorielles (à encoder en one-hot)
categorical_cols = ['PrimaryPropertyType']

# Créez un préprocesseur pour les caractéristiques numériques
numeric_transformer = StandardScaler()

# Préprocesseur pour les colonnes catégorielles (encodage en one-hot)
categorical_transformer = OneHotEncoder(drop='first')


# Créez une fonction pour entraîner et évaluer un modèle
def train_evaluate_model(model, X_train, y_train, X_test, y_test):
    
    # Créer un pipeline avec le préprocesseur et le modèle
    # Définition du ColumnTransformer
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    # Entraîne le modèle via le pipeline
    pipeline.fit(X_train, y_train)

    # Fait des prédictions sur les données de test
    y_pred = pipeline.predict(X_test)

    # Évalue la performance du modèle
    mse = mean_squared_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)

    return mse, r_squared

# Créer une liste d'algorithmes à tester
models_to_test = [
    LinearRegression(),
    ElasticNet(alpha=0.1, l1_ratio=0.5),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    SVR()
]

# Boucle à travers les modèles, entraînements et évaluations
for model in models_to_test:
    mse, r_squared = train_evaluate_model(model, X_train, y_train, X_test, y_test)
    print(f"Model: {model.__class__.__name__}")
    print("Mean Squared Error:", mse)
    print("R-squared:", r_squared)
    print()

Model: LinearRegression
Mean Squared Error: 150691.68405575003
R-squared: 0.7623781698027438

Model: ElasticNet
Mean Squared Error: 265730.4350402563
R-squared: 0.5809765302641503

Model: RandomForestRegressor
Mean Squared Error: 91242.15376812387
R-squared: 0.8561226008895095

Model: GradientBoostingRegressor
Mean Squared Error: 127046.40793112261
R-squared: 0.7996637959038814

Model: SVR
Mean Squared Error: 641167.8570561055
R-squared: -0.011041057852661407

